In [1]:
def verbose_klines(stage, klines):
    print('As últimas klines para a análise de {} são:'.format(stage))
    print(' ')
    print(klines.tail())
    print(' ')

def verbose_position(stage, position):
    print('A posição atual é: {}'.format(position.show()))
    print(' ')

In [8]:
#Trader v0.2.0
from modules.assets_handler import *
from pydoc import locate
from modules.klines_handler import OneMinuteCandlesAmount as OMCA, Transform

class Trader:
    # Common instantiation, trader and backing test
    def __init__(self, exchange_name, asset_symbol):
        # Asset Stuff
        self.exchange_name = exchange_name
        self.asset_symbol = asset_symbol
        self.Asset = AssetsHandler(exchange_name, asset_symbol)
        # Strategy
        self.strategy_name = self.Asset.operational_parameters()['strategy']['name']
        self.strategy_param = self.Asset.operational_parameters()['strategy']['parameters']
        self.strat_candle_interval = self.strategy_param['candle_interval']
        self.Strategy = locate('modules.strategies.' + self.strategy_name)(self.strategy_param)        
        self.StrategyCandlesNum = self.Strategy.how_many_candles()        
        # Stop Loss
        self.stop_name = self.Asset.operational_parameters()['stop_loss']['name']
        self.stop_param = self.Asset.operational_parameters()['stop_loss']['parameters']
        self.stop_candle_interval = self.stop_param['candle_interval']
        self.StopLoss = locate('modules.stop_loss.' + self.stop_name)(self.stop_param)
        self.StopCandlesNum = self.StopLoss.how_many_candles()
        # Order Handler (broker)
        self.OrderHandler = locate('modules.order_handler' + exchange_name.capitalize() + 'Handler')
        # Klines Handler
        self.KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() + 'FromDb')()
        
    def for_real(self):
        self.position = RealTradePosition(self.Asset.get_dataframe().iloc[0].position)
    
    def backing_test(self):
        self.position = BackingTestPosition({'side': 'closed', 'size': 0.0, 'target_price': 0.0})
        
        print('Loanding klines...')
        print(' ')
        all_1m_klines = self.KlinesHandler.all_latest_one_minute(self.asset_symbol)
        print ('Ok! klines load!')
        print(' ')
        
        #TODO: Verificar se os tamanhos das 'slices' são menores que o número de klines de 1m
        strategy_1m_slice = OMCA(self.strat_candle_interval).total(self.StrategyCandlesNum)
        stop_1m_slice = OMCA(self.stop_candle_interval).total(self.StopCandlesNum)
        
        end_index = max(strategy_1m_slice, stop_1m_slice)
        strategy_start_index = end_index - strategy_1m_slice; stop_start_index = end_index - stop_1m_slice
        
        print('Start to cycle')
        print(' ')
        while end_index < len(all_1m_klines):
            verbose_position('backing test', self.position)
            
            if(self.position.side != 'closed'):
                print('You are positioned! Checking stop')
                print(' ')
                
                klines_to_stop_analisys = Transform(
                    all_1m_klines[stop_start_index:end_index]).from_1m_to(self.stop_candle_interval)
                
                verbose_klines('Stop', klines_to_stop_analisys)
                
                stop = self.StopLoss.verify(klines_to_stop_analisys, self.position)
    
                if (stop.is_true):
                    print('Stop hit!')
                    print(' ')
                    order = self.OrderHandler.execute(stop.order, in_mode='backing_test')
                    order_confirmation = self.OrderHandler.check(order, in_mode='backing_test')
                    position.update_with(order_confirmation)
                
                else:
                    print('Dont stop for now')
                    print(' ')
    
                if(stop.update_target):
                    print('New target price hit!')
                    print(' ')
                    
                    self.position.target_price = stop.new_target_price
            
            klines_to_strategy_analisys = Transform(
                all_1m_klines[strategy_start_index:end_index]).from_1m_to(self.strat_candle_interval)
            verbose_klines('Strategy', klines_to_strategy_analisys)

            trade = self.Strategy.verify(klines_to_strategy_analisys, self.position)

            if (trade.is_true):
                print('Trade!')
                print(' ')
                order = self.OrderHandler.execute(trade.order, in_mode='backing_test')
                order_confirmation = self.OrderHandler.check(order, in_mode='backing_test')
                self.position.update_with(order_confirmation)
            
            strategy_start_index+=1; stop_start_index+=1; end_index+=1

In [9]:
exchange_name, asset_symbol = 'binance', 'BTCUSDT'

trader = Trader(exchange_name, asset_symbol)

In [10]:
trader.backing_test()

Loanding klines...
 
Ok! klines load!
 
Start to cycle
 
A posição atual é: {'side': 'closed', 'size': 0.0, 'target_price': 0.0}
 
As últimas klines para a análise de Strategy são:
 
              open_time     open     high      low    close   volume
95  2019-09-05 00:00:00  10565.9  10575.2  10487.2  10508.5  2093.31
96  2019-09-05 01:00:00    10509  10554.8  10494.3    10533  926.797
97  2019-09-05 02:00:00    10533  10558.9  10502.7    10536  815.014
98  2019-09-05 03:00:00  10537.1  10563.2    10490  10550.4  1067.49
99  2019-09-05 04:00:00    10550  10608.7  10539.3  10572.4  1105.95
 


KeyError: -1

In [ ]:
#'order', tanto no stop quanto na estratégia, será um dicionário, para manter a simplicidade

In [ ]:
exchange_name, asset_symbol = 'binance', 'BTCUSDT'

trader = Trader(exchange_name, asset_symbol)

In [ ]:
trader.backing_test()

In [ ]:
trader.position.show()

In [ ]:
trader.strategy_1m_slice

In [ ]:
from modules.klines_handler import OneMinuteCandlesAmount, Transform

n_stop = OneMinuteCandlesAmount(trader.stop_param['candle_interval']).total(trader.StopLoss.how_many_candles())

In [ ]:
n_stop

In [17]:
exchange_name = 'binance'
asset_symbol = 'BTCUSDT'

from modules.assets_handler import AssetsHandler
from modules.klines_handler import OneMinuteCandlesAmount, Transform
from pydoc import locate
import time

Asset = AssetsHandler(exchange_name, asset_symbol)

#Operational Parameters
op_param = Asset.operational_parameters()
strategy_param = op_param['strategy']['parameters']
stop_param = op_param['stop_loss']['parameters']

strategy_name = op_param['strategy']['name']
stop_name = op_param['stop_loss']['name']

Strategy = locate('modules.strategies.' + strategy_name)(strategy_param)
StopLoss = locate('modules.stop_loss.' + stop_name)(stop_param)
OrderHandler = locate('modules.order_handler' + exchange_name.capitalize() + 'Handler')
KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() + 'FromDb')()


In [20]:
position = RealTradePosition(Asset.get_dataframe().iloc[0].position)

n_candles_to_strategy_analisys = Strategy.how_many_candles()

klines_to_strategy_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                       candle_interval=strategy_param['candle_interval'], 
                                                       number_of_candles=n_candles_to_strategy_analisys)

trade = Strategy.verify(klines_to_strategy_analisys, position)

print('Últimas klines')
print(' ')
print(klines_to_strategy_analisys.tail())
print(' ')
print('Posição atual.....................: ', Asset.info().position.iloc[0]['side'])
print('Preço alvo Atual..................: ', Asset.info().position.iloc[0]['target_price'])
print('Posição e alavancagem definidos....: ', Strategy.what_side_and_leverage(klines_to_strategy_analisys))
print('Trade.............................: ', trade.is_true)
print('Ordem dada........................: ', trade.command)
print('Só para teste, qual a alavancagem.: ', trade.leverage)

KeyError: -1

In [14]:
op_param = Asset.get_dataframe()

In [15]:
op_param.op_param

,asset_symbol,auto_update,status,last_updated_by_pid,collect_data_since,trade_on,available_amount,operational_parameters,position
0,BTCUSDT,ON,full,5037,2019-09-01 00:00:00+00:00,N,0.0,"{'strategy': {'name': 'CrossSMA', 'parameters'...","{'side': 'closed', 'size': 0.0, 'target_price'..."


In [ ]:

from modules.klines_handler import *

# Teste estratégia

n_candles_to_strategy_analisys = Strategy.how_many_candles()

klines_to_strategy_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                       candle_interval=strategy_param['candle_interval'], 
                                                       number_of_candles=n_candles_to_strategy_analisys)

trade = Strategy.verify(klines_to_strategy_analisys, Asset.info().position.iloc[0])

print('Últimas klines')
print(' ')
print(klines_to_strategy_analisys.tail())
print(' ')
print('Posição atual.....................: ', Asset.info().position.iloc[0]['side'])
print('Preço alvo Atual..................: ', Asset.info().position.iloc[0]['target_price'])
print('Posição e alavancagem definidos....: ', Strategy.what_side_and_leverage(klines_to_strategy_analisys))
print('Trade.............................: ', trade.is_true)
print('Ordem dada........................: ', trade.command)
print('Só para teste, qual a alavancagem.: ', trade.leverage)

In [ ]:
trader.Strategy.how_many_candles()

In [ ]:
test_klines = trader.KlinesHandler.get_latest(asset_symbol, '1m', 100)
test_klines

In [ ]:
trader.stop_param['candle_interval']

In [ ]:
from modules.klines_handler import OneMinuteCandlesAmount, Transform

In [ ]:
OneMinuteCandlesAmount('1h').total(100)

In [ ]:
trader.Asset.get_dataframe().iloc[0].position

In [ ]:
trader.backing_test()

In [ ]:
StopCandlesNum = 'test'
StrategyCandlesNum
n_candles_to_stop_analisys = 

In [ ]:
trader.KlinesHandler.all_latest_one_minute

In [ ]:
class Myclass:
    def __init__(self):
        self.my_name_is = self.__class__.__name__
    
    def who_amn_i(self):
        print('hello',self.my_name_is)

In [ ]:
Myclass().who_amn_i()

In [ ]:
some_function('marcus')

In [ ]:
my_function.__name__

In [ ]:
marcus.__name__

In [ ]:
class Outer:
    
    def __init__(Out, param_to_outer):
        Out.param_to_outer = param_to_outer
    
    def print_outer_exclusive(Out):
        print('Hy, im outer', Out.param_to_outer)
    
    class Inner(Outer):
        
        def __init__(self, param_to_inner):
            self.param_to_inner = param_to_inner
        
        def print_inner_exclusive(self):
            print('Hy, im inner', self.param_to_inner)
        
        def print_out_and_inner(self):
            print('Hy, im double printer. This is the inner arg: {}, and this is the out arg {}'.format(
            self.param_to_inner, Outer('test').print_outer_exclusive()))

In [ ]:
Outer('olá').print_outer_exclusive()

In [ ]:
Outer('olá').Inner('paraminner').print_inner_exclusive()

In [ ]:
Outer('olá').Inner('paraminner').print_out_and_inner()